In [8]:
from config import *
import tweepy
import psycopg2
import getpass
import json
from datetime import datetime as dt

p = 'dataisgreat'

# p = getpass.getpass(prompt="Password: ")
# conn = psycopg2.connect(user = "postgres",
#                                   password = f"{p}",
#                                   host = "localhost",
#                                   port = "5432",
#                                   database = "twitter_db")

conn = psycopg2.connect(user = "postgres",
                                  password = f"{p}",
                                  host = "127.0.0.1",
                                  port = "3306",
                                  database = "postgres")

cursor = conn.cursor()

# cursor.execute("select * from twitter_feed")

# fetch = cursor.fetchone()
# print(fetch)
# rds_connection_string = f"postgres:{p}@localhost:5432/twitter_db"
# engine = create_engine(f'postgresql://{rds_connection_string}')

In [9]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
try:
    api.verify_credentials()
    print("Authentication OK")
except:
    print("Error during authentication")

my_list = []
class MyStreamListener(tweepy.StreamListener):
    def __init__(self, api=None):
        super(MyStreamListener, self).__init__()
        self.num_tweets = 0
    
    def on_status(self, status):
        collection = []
        print(status.text)
        #Don't know why it doesnt print out coordinates
        print(status.coordinates)
        self.num_tweets += 1
        if self.num_tweets < 20:
            collection.append(status)
            return True
        else:
            return False
        
    def on_data(self, data):
        print("On Data")
        all_data = json.loads(data)
#         print(json.dumps(all_data, indent=4))
        date = all_data['created_at']
        place = all_data['user']['location']
        tweet = all_data["text"].replace('"',"")
        username = all_data["user"]["screen_name"]
        metadata = all_data['user']["favourites_count"]
        print(date)
        print((place,username,tweet))
        self.num_tweets += 1
        if self.num_tweets < 500:
            cursor.execute('INSERT INTO twitter_feed (date, user_location, user_id, text, metadata) \
            VALUES (%s,%s,%s,%s,%s);', (dt.strptime(date,'%a %b %d %H:%M:%S %z %Y').strftime('%Y-%m-%d %H:%M:%S'),place,tweet,username,metadata))
            
            cursor.execute('INSERT INTO tweets (data) VALUES (%s);', (json.dumps(all_data),))
            conn.commit()
            
            print(self.num_tweets)
            print((username,tweet))

            return True
        else:
            return False
    
        


myStreamListener = MyStreamListener()

myStream = tweepy.Stream(auth=api.auth, listener=myStreamListener)


myStream.filter(track=['election2020'], is_async=True)

Authentication OK
On Data
Sat Feb 22 17:42:05 +0000 2020
('Oklahoma', 'Nancy31Suh', 'RT @Todd_Michael_: NOW HIRING -Trump Supporters! 💵Image is EVERYTHING. They have to keep the illusion ALIVE to intimidate the electorate an…')
1
('Nancy31Suh', 'RT @Todd_Michael_: NOW HIRING -Trump Supporters! 💵Image is EVERYTHING. They have to keep the illusion ALIVE to intimidate the electorate an…')
On Data
Sat Feb 22 17:42:16 +0000 2020
('San Diego, CA', 'jay_snowdon', '@ChidiNwatu @realDonaldTrump Ruskie support for Bernie because Bernie helps Trump win in November. Russians underst… https://t.co/gztSseyPMl')
2
('jay_snowdon', '@ChidiNwatu @realDonaldTrump Ruskie support for Bernie because Bernie helps Trump win in November. Russians underst… https://t.co/gztSseyPMl')
On Data
Sat Feb 22 17:42:17 +0000 2020
(None, 'LindaBorland5', 'RT @williewjewell: Florida @GovRonDeSantis is requiring that all persons seeking employment in the State be confirmed e-verified legal resi…')
3
('LindaBorland5', 'RT @w

On Data
Sat Feb 22 17:44:06 +0000 2020
(None, 'diachkova_nina', 'RT @hankboonelife: I’m endorsing @VancityReynolds for Vice President. #Election2020')
24
('diachkova_nina', 'RT @hankboonelife: I’m endorsing @VancityReynolds for Vice President. #Election2020')
On Data
Sat Feb 22 17:44:13 +0000 2020
('Austin, TX', 'RascalPinkish', "So let's get this straight.. the #RussianElectionInterference news comes out and Bernie Sanders responds by.. attac… https://t.co/fXW1hwugre")
25
('RascalPinkish', "So let's get this straight.. the #RussianElectionInterference news comes out and Bernie Sanders responds by.. attac… https://t.co/fXW1hwugre")
On Data
Sat Feb 22 17:44:15 +0000 2020
('United States', 'Road_trippn', 'RT @Todd_Michael_: NOW HIRING -Trump Supporters! 💵Image is EVERYTHING. They have to keep the illusion ALIVE to intimidate the electorate an…')
26
('Road_trippn', 'RT @Todd_Michael_: NOW HIRING -Trump Supporters! 💵Image is EVERYTHING. They have to keep the illusion ALIVE to intimidate th

In [3]:
date ='Fri Feb 21 03:36:01 +0000 2020'

In [4]:
date

'Fri Feb 21 03:36:01 +0000 2020'

In [5]:
type(dt.strptime(date,'%a %b %d %H:%M:%S %z %Y').strftime('%Y-%m-%d %H:%M:%S'))

str

In [10]:
cursor.execute("select * from tweets")

fetch = cursor.fetchone()
print(json.dumps(fetch,indent=4))

[
    1,
    {
        "id": 1230732175042961408,
        "geo": null,
        "lang": "en",
        "text": "RT @BbyDmplng403: @LouiseLowrey There are so many hilarious comments in You Tube @PatriotsSoapbox chat on @_saltycrystal show tonight.",
        "user": {
            "id": 1080974939446861825,
            "url": null,
            "lang": null,
            "name": "Sonia Hill",
            "id_str": "1080974939446861825",
            "location": null,
            "verified": false,
            "following": null,
            "protected": false,
            "time_zone": null,
            "created_at": "Thu Jan 03 23:51:23 +0000 2019",
            "utc_offset": null,
            "description": null,
            "geo_enabled": false,
            "screen_name": "SoniaHi27303526",
            "listed_count": 0,
            "friends_count": 1034,
            "is_translator": false,
            "notifications": null,
            "statuses_count": 23058,
            "default_profile": t

On Data
Sat Feb 22 17:46:09 +0000 2020
(None, 'POC4Tulsi', 'RT @GlobeOpinion: .@stephenkinzer is voting for @TulsiGabbard. Here’s why. #Election2020 https://t.co/qWTyaAdOoJ')
47
('POC4Tulsi', 'RT @GlobeOpinion: .@stephenkinzer is voting for @TulsiGabbard. Here’s why. #Election2020 https://t.co/qWTyaAdOoJ')
On Data
Sat Feb 22 17:46:13 +0000 2020
('Out of this world, earth', 'ChiefCovfefe', "RT @AlanGrayson: Russia commits a brutal cyberattack against the country of Georgia.  https://t.co/aFV5qzKZ9R Putin's Puppet DJT responds b…")
48
('ChiefCovfefe', "RT @AlanGrayson: Russia commits a brutal cyberattack against the country of Georgia.  https://t.co/aFV5qzKZ9R Putin's Puppet DJT responds b…")
On Data
Sat Feb 22 17:46:17 +0000 2020
('Redmond, WA, USA', 'MSN', 'Follow here for the latest on the Nevada caucuses, plus polls, candidate bios and state-by-state developments.… https://t.co/VzLTCsPGWq')
49
('MSN', 'Follow here for the latest on the Nevada caucuses, plus polls, candidate bios and 

On Data
Sat Feb 22 17:49:21 +0000 2020
('Somewhere in California', 'mousdrvr', 'RT @SalKappa: #PresidentSanders\n#BernieWonIowa #BernieWonNewHampshire\n#BernieBeatsTrump\n\n#BerniesGonnaWin\n#NevadaCaucus #SouthCarolina\n\n#Fe…')
72
('mousdrvr', 'RT @SalKappa: #PresidentSanders\n#BernieWonIowa #BernieWonNewHampshire\n#BernieBeatsTrump\n\n#BerniesGonnaWin\n#NevadaCaucus #SouthCarolina\n\n#Fe…')
On Data
Sat Feb 22 17:49:23 +0000 2020
('Washington DC', 'ASALH', 'RT @robbiethompson: .@K_JeanPierre discusses the power of the vote and the fight for political power at the 2020 @ASALH #BlackHistoryLunche…')
73
('ASALH', 'RT @robbiethompson: .@K_JeanPierre discusses the power of the vote and the fight for political power at the 2020 @ASALH #BlackHistoryLunche…')
On Data
Sat Feb 22 17:49:28 +0000 2020
('Minneapolis', 'Retrievals1', '👇')
74
('Retrievals1', '👇')
On Data
Sat Feb 22 17:49:28 +0000 2020
('California', 'PeauNue', 'RT @LizMacDonaldFOX: Your thoughts?')
75
('PeauNue', 'RT @LizMacDonaldF

On Data
Sat Feb 22 17:51:39 +0000 2020
('United States, NorCal', 'RobbinMilne', "Actually, not factual at all. You're a shill spreading #propaganda - #mediabias rampant this #Election2020 cycle &amp;d… https://t.co/dHoOC9djsq")
98
('RobbinMilne', "Actually, not factual at all. You're a shill spreading #propaganda - #mediabias rampant this #Election2020 cycle &amp;d… https://t.co/dHoOC9djsq")
On Data
Sat Feb 22 17:51:42 +0000 2020
('🌎WW', 'DanRidgway3', 'RT @TrumpsPitBull: Let’s flip this poll, patriots. More Christians, less ANTI American Muslims in our Congress 🙏🇺🇸❤️🐾')
99
('DanRidgway3', 'RT @TrumpsPitBull: Let’s flip this poll, patriots. More Christians, less ANTI American Muslims in our Congress 🙏🇺🇸❤️🐾')
On Data
Sat Feb 22 17:52:09 +0000 2020
('Youngstown, OH', 'DonelAdams', "RT @MSN: Follow here for the latest on today's Nevada caucuses, including polling, candidate bios and state-by-state updates. #Election2020…")
100
('DonelAdams', "RT @MSN: Follow here for the latest on today's

On Data
Sat Feb 22 17:54:45 +0000 2020
('Marietta, GA', 'DGPurser', 'RT @gailsline: Ok Christians!  We are not ashamed of the gospel and boldly proclaim our Lord and Savior Jesus.  \nLet’s flip this poll.  Ple…')
122
('DGPurser', 'RT @gailsline: Ok Christians!  We are not ashamed of the gospel and boldly proclaim our Lord and Savior Jesus.  \nLet’s flip this poll.  Ple…')
On Data
Sat Feb 22 17:55:01 +0000 2020
('Florida, USA', 'thatgirlsandra5', 'RT @WarRoom2020: Twitter suspends 70 Bloomberg-boosting accounts for violating company policy #Election2020 https://t.co/98Bb8Qvjju')
123
('thatgirlsandra5', 'RT @WarRoom2020: Twitter suspends 70 Bloomberg-boosting accounts for violating company policy #Election2020 https://t.co/98Bb8Qvjju')
On Data
Sat Feb 22 17:55:03 +0000 2020
('US and Worldwide', 'tekito_time', 'Bernie Sanders wants to convert American Capitalist corporations into socialist institutions. The new naming system… https://t.co/VviVE6LaA7')
124
('tekito_time', 'Bernie Sanders w

On Data
Sat Feb 22 17:56:16 +0000 2020
('United States', 'Pismo_B', 'RT @gailsline: Ok Christians!  We are not ashamed of the gospel and boldly proclaim our Lord and Savior Jesus.  \nLet’s flip this poll.  Ple…')
146
('Pismo_B', 'RT @gailsline: Ok Christians!  We are not ashamed of the gospel and boldly proclaim our Lord and Savior Jesus.  \nLet’s flip this poll.  Ple…')
On Data
Sat Feb 22 17:56:19 +0000 2020
('michigan usa', 'maddmav', 'RT @dewmegrl: WOW.. 😱😱🚨 👉American flag is REMOVED before debate begins. 💥This is a slap in the face to every soldier who died defending thi…')
147
('maddmav', 'RT @dewmegrl: WOW.. 😱😱🚨 👉American flag is REMOVED before debate begins. 💥This is a slap in the face to every soldier who died defending thi…')
On Data
Sat Feb 22 17:56:22 +0000 2020
('Southern California', 'KarlEspo', 'RT @ChristLawmakers: Do you believe America would be better off if more Christians served in elected office?\n#Retweet #Poll #Election2020 #…')
148
('KarlEspo', 'RT @ChristLawmake